# Filtracja Non-Local Means

## Definicja

Kolejny "poziom wtajemniczenia" w zagadnienie filtracji obrazów to metoda Non-Local Means (NLM).
Została ona zaproponowana w pracy *A non-local algorithm for image denoising* autorstwa Antoni Buades, Bartomeu Coll, i Jean Michel Morel na konferencji CVPR w 2005 roku.

Filtr NLM dany jest zależnością:

\begin{equation}
\hat{I}(\mathbf{x}) = \sum_{\mathbf{p} \in V(\mathbf{x})} w(\mathbf{p},\mathbf{x})I(\mathbf{p})
\end{equation}

gdzie:
- $I$ - obraz wejściowy,
- $\hat{I}$ - obraz wyjściowy (przefiltrowany),
- $\mathbf{x}$ - współrzędne piksela obrazu,
- $V(\mathbf{x})$ - obszar poszukiwań piksela, dla którego przeprowadzana jest filtracja,
- $w$ - waga punktu $\mathbf{p}$ z obszaru poszukiwań.

Wróćmy na chwilę do filtracji bilateralnej. Tam waga danego piksela z kontekstu zależała od dwóch czynników - odległości przestrzennej pomiędzy pikselami oraz różnicy w jasności/kolorze pomiędzy pikselami (tzw. przeciwdziedzina).
Filtr NLM stanowi uogólnienie tej metody - do obliczania wag nie wykorzystuje się już pojedynczych pikseli ($\mathbf{p}$ i $\mathbf{x}$), a lokalne konteksty ($N(\mathbf{p})$ i $N(\mathbf{x})$).

Waga $w$ dana jest następującą zależnością:

\begin{equation}
w(\mathbf{p},\mathbf{x}) = \frac{1}{Z(\mathbf{x})}\exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\end{equation}

gdzie:
- \begin{equation}
Z(\mathbf{x}) = \sum_{\mathbf{p} \in  V(\mathbf{x})} \exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\end{equation},
- $|| \cdot ||$ - jest normą $L_2$ odległości pomiędzy dwoma kontekstami,
- $v$ oznacza mnożenie punktowe kontekstu $N$ przez dwuwymiarową maskę Gaussa o odpowiadających kontekstowi wymiarach,
- $\alpha$ > 0 - parametr sterujący filtracją,
- $\sigma$ - parametr szumu stacjonarnego występującego na obrazie (w przypadku szumu niestacjonarnego, parametr $\sigma$ musi zostać dopasowany lokalnie tj. $\sigma = \sigma(\mathbf{x})$).

## Analiza działania

Zastanówmy sie teraz jak działa filtra NLM. Najprościej to zrozumieć na rysunku.



1. Dla rozważanego piksela $\mathbf{x}$ definiujemy obszar poszukiwań $V(\mathbf{x})$. Uwaga - obszar poszukiwań ($V$) jest jednostką większą niż otoczenie/kontekst ($N$).

2. Następnie, dla każdego z pikseli $\mathbf{p} \in  V(\mathbf{x})$ oraz samego $\mathbf{x}$ definiujemy otoczenie/kontekst odpowiednio $N(\mathbf{p})$ i $N(\mathbf{x})$.

3. Wracamy do równania definiującego wagę  $w(\mathbf{p},\mathbf{x})$, a konkretnie do wyrażenia $|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||$. Przeanalizujmy co ono oznacza. Mamy dwa otoczenia: $N(\mathbf{p})$ i $N(\mathbf{x})$. Każde z nich mnożymy przez odpowiadającą maskę Gaussa - funkcja $v$. Otrzymujemy dwie macierze, które odejmujemy od siebie punktowo. Następnie obliczamy kwadrat z normy ($L_2$ definiujemy jako $||X||_2 = \sqrt{\sum_k|X_k|^2}$. Otrzymujemy zatem jedną liczbę, która opisuje nam podobieństwo otoczeń pikseli $\mathbf{x}$ i $\mathbf{p}$. Mała wartość oznacza otoczenia zbliżone, duża - różniące się. Ponieważ, z dokładnością do stałych, liczba ta stanowi wykładnik funkcji $e^{-x}$, to ostatecznie waga jest zbliżona do 1 dla otoczeń podobnych, a szybko maleje wraz z malejącym podobieństwem kontekstów.

4. Podsumowując. Jak wynika z powyższej analizy filtr NLM to taki filtr bilateralny, w którym zamiast pojedynczych pikseli porównuje się ich lokalne otoczenia. Wpływa to pozytywnie na jakość filtracji, niestety kosztem złożoności obliczeniowej.

## Implementacja

W ramach zadania należy zaimplementować filtr NLM, ocenić jego działanie w porównaniu do filtra Gaussa i bilateralnego oraz dokonać pomiaru czasu obliczeń (dla trzech wymienionych metod).

Jak już się zrozumie jak działa NLM, jego implementacja jest dość prosta.
Wartość parametru $\alpha$ należy dobrać eksperymentalnie.
Nie należy także "przesadzić" z rozmiarem obszaru poszukiwań (np. 11x11) oraz kontekstu (5x5 lub 3x3).

Wskazówki do implementacji:
- algorytm sprowadza się do dwóch podwójnych pętli for: zewnętrzne po pikselach, wewnętrzne po kolejnych obszarach przeszukań,
- przed realizacją trzeba przemyśleć problem pikseli brzegowych - de facto problemów jest kilka. Po pierwsze nie dla każdego piksela można wyznaczyć pełny obszar przeszukań (tu propozycja, aby filtrację przeprowadzać tylko dla pikseli z pełnym obszarem). Po drugie, ponieważ rozpatrujemy konteksty, to nawet dla piksela o "pełnym" obszarze przeszukań, będą istnieć piksele, dla których nie pełnych kontekstów (sugestia - powiększyć obszar przeszukać, tak aby zawierał konteksty). Ostatni problem jest bardziej techniczny/implementacyjny. Jeśli w kolejnych iteracjach "jawnie" wytniemy fragment o rozmiarach obszaru przeszukiwań, to znowu pojawi się problem brzegowy - tu można albo wyciąć nieco większy obszar, albo cały czas "pracować" na obrazie oryginalnym ("żonglerka indeksami").
- warto sprawdzać indeksy i rozmiary "wycinanych" kontekstów,
- wagi wyliczamy w trzech krokach:
    - obliczenia dla $N(\mathbf{x})$ + inicjalizacja macierzy na wagi,
    - podwójna pętla, w której przeprowadzamy obliczenia dla kolejnych $N(\mathbf{p})$ oraz wyliczamy wagi,
    - normalizacja macierzy wag oraz końcowa filtracja obszaru w wykorzystaniem wag.
- uwaga, obliczenia trochę trwają, nawet dla obrazka 256x256 i względnie niewielkich obszaru przeszukań i kontesktu.

Efekt końcowy:
- porównanie wyników metod: filtr Gaussa, filtr bilateralny oraz filtr NLM (2-3 zdania komentarza),
- porównanie czasu działania powyższych metod (1 zdanie komentarza).


In [1]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os
from scipy import signal
import cv2
import copy
from numpy import linalg as LA
import time

if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate
        
mat = loadmat('MR_data.mat')

Input_1 = mat['I_noisy1']
Input_2 = mat['I_noisy2']
Input_3 = mat['I_noisy3']
Input_4 = mat['I_noisy4']
Input_0 = mat['I_noisefree']

In [2]:
def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum() 

In [3]:
def fgaussian_bez(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g 

In [4]:
def NLM(img,window,area,alfa,sigma_square):
    v=fgaussian(window,np.sqrt(sigma_square))
    IConvolucja = img.copy()
    (X,Y)=IConvolucja.shape
    area_half = area//2
    window_half = window//2
    for i in range(0+area_half+window_half,X-area_half-window_half):
        for j in range(0+area_half+window_half,Y-area_half-window_half):
                i_minus=i-area_half-window_half
                i_plus=i+area_half+window_half+1
                j_minus=j-area_half-window_half
                j_plus=j+area_half+window_half+1
                obszar = img[i_minus:i_plus,j_minus:j_plus]
                a,b = obszar.shape
                x = [i,j]
                w_px=0
                Zx=0
                new_pixel=0
                for k in range(0+window_half,a-window_half):
                    for l in range(0+window_half,b-window_half):
                        N_p=obszar[k-window_half:k+window_half+1,l-window_half:l+window_half+1]
                        N_x=img[i-window_half:i+window_half+1,j-window_half:j+window_half+1]
                        v_Np = v*N_p
                        v_Nx=v*N_x
                        Xk = v_Np-v_Nx
                        X__2 = np.sqrt((Xk**2).sum())
                        wpx = np.exp((-(X__2**2))/(alfa*sigma_square))
                        Zx = Zx + wpx
                        new_pixel = new_pixel + wpx*obszar[k,l]
                new_pixel=new_pixel/Zx
                IConvolucja[i,j]=new_pixel
    return IConvolucja         

In [5]:
def pixel_out(okno,filtr,variancy):
    A,B = okno.shape
    pixel=0
    x = [A//2,B//2]
    for i in range(A):
        for j in range(B):
            AB=[i,j]
            
            y=np.sqrt(((x[0]-AB[0])**2)+((x[1]-AB[1])**2))
            gaus= np.exp(-(y**2)/(2*(variancy**2)))
            pixel=pixel+gaus*okno[i,j]
    pixel=pixel/filtr.sum()
    return pixel

In [6]:
#TODO Samodzielna
def convol(img,window,variancy):
    filtr=fgaussian_bez(5,variancy)
    IConv = img.copy()
    (X,Y)=IConv.shape
    polowa = window//2
    for i in range(0+window//2,X-window//2):
        for j in range(0+window//2,Y-window//2):
            okno = IConv[i-polowa:i+polowa+1,j-polowa:j+polowa+1]
            new_pixel=pixel_out(okno,filtr,variancy)
            IConv[i,j]=new_pixel
    return IConv


In [7]:
def pixel_out_2(okno,filtr,variancy,delta_r):
    A,B = okno.shape
    pixel=0
    normalization=0
    x = [A//2,B//2]
    for i in range(A):
        for j in range(B):
            AB=[i,j]
            
            y=np.sqrt(((x[0]-AB[0])**2)+((x[1]-AB[1])**2))
            gaus= np.exp(-(y**2)/(2*(variancy**2)))
            
            diff=np.abs(okno[A//2,B//2]-okno[i,j])
            gaus_diff= np.exp(-(diff**2)/(2*(delta_r**2)))
            
            
            pixel=pixel+gaus*gaus_diff*okno[i,j]
            normalization+=gaus*gaus_diff
    pixel=pixel/(normalization)
    return pixel

In [8]:
#TODO Samodzielna
def bilateral(img,window,variancy,delta_r):
    filtr=fgaussian_bez(window,variancy)
    IConvolucja = img.copy()
    (X,Y)=IConvolucja.shape
    polowa = window//2
    for i in range(0+window//2,X-window//2):
        for j in range(0+window//2,Y-window//2):
            okno = IConvolucja[i-polowa:i+polowa+1,j-polowa:j+polowa+1]
            new_pixel=pixel_out_2(okno,filtr,variancy,delta_r)
            IConvolucja[i,j]=new_pixel
    return IConvolucja




Do porównania zostanie użyty najbardziej zaszumiony obraz, czyli u mnie Input_2.

Pozwoli to najlepiej zoobrazować różnice pomiędzy algorytmami

In [9]:
start_NLM = time.time()
test_NLM=NLM(Input_2,window=3,area=7,alfa=5,sigma_square=50)
stop_NLM = time.time()
time_past_NLM = stop_NLM - start_NLM

In [11]:
window=5
variancy=0.9

delta_r=45
variancy_1=3
start_conv=time.time()
convolucja_2=convol(Input_2,window,variancy)
stop_conv=time.time()
time_past_conv = stop_conv - start_conv
start_bilateral=time.time()
bilateralne_2=bilateral(Input_2,window,variancy_1,delta_r)
stop_bilateral=time.time()
time_past_bilateral = stop_bilateral - start_bilateral

"""
#Proszę sobie puścić bo nie przejdzie przez upla ze zdjęciami
f, ax1 = plt.subplots(2,2,figsize=(16,16))
ax1[0,0].imshow(convolucja_2, 'gray')
ax1[0,0].set_title("Konwolucja - czas - {}".format(time_past_conv))
ax1[0,0].axis('off')
ax1[0,1].imshow(bilateralne_2, 'gray')
ax1[0,1].set_title("Filtracja bilateralna - czas - {}".format(time_past_bilateral))
ax1[0,1].axis('off')
ax1[1,0].imshow(test_NLM, 'gray')
ax1[1,0].set_title("Filtracja NLM - czas - {}".format(time_past_NLM))
ax1[1,0].axis('off')
ax1[1,1].imshow(Input_2, 'gray')
ax1[1,1].set_title("Oryginał")
ax1[1,1].axis('off')
"""

'\n#Proszę sobie puścić bo nie przejdzie przez upla ze zdjęciami\nf, ax1 = plt.subplots(2,2,figsize=(16,16))\nax1[0,0].imshow(convolucja_2, \'gray\')\nax1[0,0].set_title("Konwolucja - czas - {}".format(time_past_conv))\nax1[0,0].axis(\'off\')\nax1[0,1].imshow(bilateralne_2, \'gray\')\nax1[0,1].set_title("Filtracja bilateralna - czas - {}".format(time_past_bilateral))\nax1[0,1].axis(\'off\')\nax1[1,0].imshow(test_NLM, \'gray\')\nax1[1,0].set_title("Filtracja NLM - czas - {}".format(time_past_NLM))\nax1[1,0].axis(\'off\')\nax1[1,1].imshow(Input_2, \'gray\')\nax1[1,1].set_title("Oryginał")\nax1[1,1].axis(\'off\')\n'

Jak widać najlepszy wynik osiągnęła ostatnia metoda

Udało jej się zlikwidować cały szum a jednocześnie krawędzie dalej są widoczne i odróżnialne

Widać również że filtr bilateralny jest znacząco lepszy od filtru gaussa dzięki lepszemu zachowaniu występujących krawędzi

Nie zmienia to faktu że każda kolejna metoda wykorzystuje coraz więcej zasobów i potrzebuje więcej pamięci

Widać to idealnie w czasie wykonywania funkcji, która dla metody NLM trwa ponad 4 razy dłużej niż dla gaussa i około 2.5 raza dłużej niż dla filtracji bilateralnej